In [ ]:
import cv2
import os
import numpy as np
from tqdm import tqdm
from zipfile import ZipFile
from sklearn.decomposition import PCA

In [ ]:
#let's mount mt gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define your data and output directories
DATA_DIR = '/content/drive/MyDrive/'
TRAIN_ZIP_PATH = DATA_DIR + 'data_models/label.zip'
TRAIN_OUTPUT_DIR = DATA_DIR + 'label_preprocess/'

# Create output directories if not exists
#os.makedirs(TRAIN_OUTPUT_DIR, exist_ok=True)
#os.makedirs(TEST_OUTPUT_DIR, exist_ok=True)

In [ ]:
"""# Function to unzip all files
def unzip_files(zip_path, output_dir):
    with ZipFile(zip_path, 'r') as zip_ref:
      try:
        zip_ref.extractall(output_dir)
      except:
        pass"""

"# Function to unzip files\ndef unzip_files(zip_path, output_dir):\n    with ZipFile(zip_path, 'r') as zip_ref:\n      try:\n        zip_ref.extractall(output_dir)\n      except:\n        pass"

In [ ]:
# unzipp code when you specify how many files from beg
def unzip_files(zip_path, output_dir, num_files=None):
    with ZipFile(zip_path, 'r') as zip_ref:
        try:
            # Get the list of files in the zip archive
            file_list = zip_ref.namelist()

            # If num_files is specified, extract only the first num_files
            if num_files:
                file_list = file_list[:num_files]

            # Extract the selected files to the output directory
            zip_ref.extractall(output_dir, members=file_list)
        except Exception as e:
            print(f"Error occurred during extraction: {e}")

In [ ]:
# Get user input for the number of files to extract
num_files_to_extract = int(input("Enter the number of files to extract: "))

Enter the number of files to extract: 2500


In [ ]:
# Example usage to extract 2,500 files from the test zip file
unzip_files(TEST_ZIP_PATH, TEST_OUTPUT_DIR, num_files=num_files_to_extract)

In [ ]:
# Function to perform cropping with a limit on the number of images
def crop_images(input_dir, output_dir, num_images=None):
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # If num_images is specified, limit the list to the first num_images
    if num_images:
        image_files = image_files[:num_images]

    # Define padding size
    padding_size = 0.2

    for image_file in tqdm(image_files, desc=f'Cropping images in {input_dir}'):
        image_path = os.path.join(input_dir, image_file)

        # Load the image
        im = cv2.imread(image_path)
        im2 = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
        ret, thresh = cv2.threshold(im2, 10, 255, 0)
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

        # Region of interest
        ROI = (0, 0, 0, 0)
        ROI_area = 0
        for contour in contours:
            x, y, width, height = cv2.boundingRect(contour)
            area = width * height
            if area > ROI_area:
                ROI_area = area
                ROI = (x, y, width, height)

        x, y, width, height = ROI

        if width > height:
            pad = int(width * padding_size)
        else:
            pad = int(height * padding_size)

        # Crop the image
        crop = im[max(0, y - pad):min(im.shape[0], y + height + pad), max(0, x - pad):min(im.shape[1], x + width + pad)]

        # Save cropped image
        destiny = os.path.join(output_dir, image_file)
        destiny = destiny[:-3] + "png"
        cv2.imwrite(destiny, cv2.resize(crop, (128, 128), interpolation=cv2.INTER_AREA), [cv2.IMWRITE_PNG_COMPRESSION, 0])


In [ ]:
# Crop images
crop_images(TRAIN_OUTPUT_DIR+'images_training_rev1', TRAIN_OUTPUT_DIR+'cropped')

Cropping images in /content/drive/MyDrive/Galaxy_Morphology/label_preprocess/images_training_rev1: 100%|██████████| 2407/2407 [02:17<00:00, 17.47it/s]


In [ ]:
# Function for additional preprocessing steps with a limit on the number of images
def preprocess_images(input_dir, output_dir, num_images=None):
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # If num_images is specified, limit the list to the first num_images
    if num_images:
        image_files = image_files[:num_images]

    for image_file in tqdm(image_files, desc=f'Performing additional preprocessing in {input_dir}'):
        image_path = os.path.join(input_dir, image_file)

        # Load the image
        rotated_crop = cv2.imread(image_path)

        # Rotate the image
        rotated_crop = cv2.rotate(rotated_crop, cv2.ROTATE_90_COUNTERCLOCKWISE)

        # Subtract background
        gray_rotated_crop = cv2.cvtColor(rotated_crop, cv2.COLOR_BGR2GRAY)
        _, thresholded = cv2.threshold(gray_rotated_crop, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        rotated_crop[thresholded == 0] = 0

        # Perform PCA
        flattened_image = rotated_crop.reshape((-1, 3))
        pca = PCA(n_components=2) ## how did they select 125 components?
        pca_result = pca.fit_transform(flattened_image)
        pca_image = pca.inverse_transform(pca_result).reshape(rotated_crop.shape)

        # Save PCA reduced color images
        destiny=os.path.join(output_dir, image_file)
        destiny = destiny[:-3] + "png"
        print("further preprocessing saved to:", destiny)
        cv2.imwrite(destiny, rotated_crop, [cv2.IMWRITE_PNG_COMPRESSION, 0]) ##save lossless png


In [ ]:
# Perform additional preprocessing
preprocess_images(TRAIN_OUTPUT_DIR+'cropped', TRAIN_OUTPUT_DIR+'final')